In [2]:
# translate

# 수행 목록
# 중심으로 기준으로 반사
# 반사된 것의 색을 반전

import cv2
import numpy as np

def warp_affine_with_center_reflect(img, mtrx, dx, dy):
    rows, cols = img.shape[:2]

    # 이미지 중앙 좌표
    center = (cols / 2, rows / 2)
    
    # 이동 매트릭스
    move_mtrx = np.float32([
        [1, 0, dx],
        [0, 1, dy]
    ])
    
    # 중심 이동 매트릭스 (중심을 원점으로 이동)
    center_to_origin = np.float32([
        [1, 0, -center[0]],
        [0, 1, -center[1]]
    ])
    
    # 원점에서 중심으로 이동
    origin_to_center = np.float32([
        [1, 0, center[0]],
        [0, 1, center[1]]
    ])
    
    # 이동 후 반사 매트릭스 (중앙을 기준으로 반사)
    # 반사는 x축 기준으로 진행
    reflect_mtrx = np.float32([
        [-1, 0, 2 * center[0]],
        [0, 1, 0]
    ])
    
    # 반사 후 이동 매트릭스 (원점에서 이동)
    reflect_mtrx = np.dot(reflect_mtrx, move_mtrx)
    
    # 최종 매트릭스는 중심 이동 -> 반사 -> 원위치 이동
    final_mtrx = np.dot(origin_to_center, np.dot(reflect_mtrx, center_to_origin))
    
    # 이미지 변환
    transformed_img = cv2.warpAffine(img, final_mtrx[:2], (cols + dx, rows + dy), 
                                     flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
    
    return transformed_img

# 이미지 불러오기
img = cv2.imread('your_image.jpg')

# 이동 변환 정도
dx, dy = 50, 50

# 중앙 기준으로 반사 적용
result_img = warp_affine_with_center_reflect(img, None, dx, dy)

# 결과 이미지 저장 또는 표시
cv2.imwrite('transformed_image.jpg', result_img)


# ---④ 탈락된 외곽 픽셀을 원본을 반사 시켜서 보정
dst3 = cv2.warpAffine(img, mtrx, (cols+dx, rows+dy), None, \
                                cv2.INTER_LINEAR, cv2.BORDER_REFLECT)

# cv2.imshow('original', img)
# cv2.imshow('trans',dst)
# cv2.imshow('BORDER_CONSTATNT', dst2)
cv2.imshow('BORDER_FEFLECT', dst3)
cv2.waitKey(0)
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# scale_matrix

import cv2
import numpy as np

img = cv2.imread('../img/fish.jpg')
height, width = img.shape[:2]

# --① 0.5배 축소 변환 행렬
m_small = np.float32([[0.5, 0, 0],
                       [0, 0.5,0]])  
# --② 2배 확대 변환 행렬
m_big = np.float32([[2, 0, 0],
                     [0, 2, 0]])  

# 선형 보간법(linear interpolation)
# 끝점의 값이 주어졌을 때
# 그 사이에 위치한 값을 추정하기 위하여
# 직선 거리에 따라 선형적으로 계산하는 방법이다

# --③ 보간법 적용 없이 확대 축소
dst1 = cv2.warpAffine(img, m_small, (int(height*0.5), int(width*0.5)))
dst2 = cv2.warpAffine(img, m_big, (int(height*2), int(width*2)))

# --④ 보간법 적용한 확대 축소
dst3 = cv2.warpAffine(img, m_small, (int(height*0.5), int(width*0.5)), \
                        None, cv2.INTER_AREA)
dst4 = cv2.warpAffine(img, m_big, (int(height*2), int(width*2)), \
                        None, cv2.INTER_CUBIC)

# 결과 출력
cv2.imshow("original", img)
cv2.imshow("small", dst1)
cv2.imshow("big", dst2)
cv2.imshow("small INTER_AREA", dst3)
cv2.imshow("big INTER_CUBIC", dst4)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
# rotate_getmatrix

import cv2

img = cv2.imread(r'C:\ex\cvdir\insightbook.opencv_project_python-master\img\blackcat.jpg')
rows,cols = img.shape[0:2]

#---① 회전을 위한 변환 행렬 구하기
# 회전축:중앙, 각도:45, 배율:0.5
m45 = cv2.getRotationMatrix2D((rows/2,cols/2),90,1) 
m45 = cv2.getRotationMatrix2D((cols/2,rows/2),90,1) 
# 회전축:중앙, 각도:90, 배율:1.5
m90 = cv2.getRotationMatrix2D((rows/2,cols/2),90,1.5) 

def rotate_image(img, angle, scale):
    rows, cols = img.shape[:2]
    
    # 회전 행렬 생성, 회전축을 이미지 중앙으로 설정
    mtrx = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, scale)
    
    # 회전된 이미지의 새로운 크기 계산
    cos_theta = np.abs(mtrx[0, 0])
    sin_theta = np.abs(mtrx[0, 1])
    
    # 회전 후 이미지 크기
    new_cols = int(rows * sin_theta + cols * cos_theta)
    new_rows = int(rows * cos_theta + cols * sin_theta)
    
    # 새로운 회전 행렬에 대한 이동 조정
    mtrx[0, 2] += (new_cols - cols) / 2
    mtrx[1, 2] += (new_rows - rows) / 2
    
    # 회전 이미지 생성
    rotated_img = cv2.warpAffine(img, mtrx, (new_cols, new_rows), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0))
    return rotated_img

def resize_and_crop_image(bg_img, overlay_img, crop_x_offset=0, crop_y_offset=0):
    bg_rows, bg_cols = bg_img.shape[:2]
    ol_rows, ol_cols = overlay_img.shape[:2]

    # 확대된 이미지가 원본 이미지의 크기에 맞게 크롭
    if ol_rows > bg_rows or ol_cols > bg_cols:
        scale_x = ol_cols / bg_cols
        scale_y = ol_rows / bg_rows
        scale = max(scale_x, scale_y)
        
        new_ol_cols = int(bg_cols * scale)
        new_ol_rows = int(bg_rows * scale)
        
        start_x = max((new_ol_cols - ol_cols) // 2 + crop_x_offset, 0)
        start_y = max((new_ol_rows - ol_rows) // 2 + crop_y_offset, 0)
        
        # 크롭
        cropped_overlay_img = overlay_img[start_y:start_y + bg_rows, start_x:start_x + bg_cols]
    else:
        cropped_overlay_img = overlay_img
    
    return cropped_overlay_img


#---② 변환 행렬 적용
# img45 = cv2.warpAffine(img, m45,(rows,cols))
# img90 = cv2.warpAffine(img, m90,(rows,cols))

roatate=rotate_image(img, 90, 1)
roatate_big=rotate_image(img, 90, 1.8)
rotate_big_edit=resize_and_crop_image(roatate, roatate_big, crop_x_offset=100, crop_y_offset=0)

#---③ 결과 출력
cv2.imshow('origin',img)
# cv2.imshow("45", img45)
cv2.imshow("45", roatate)
cv2.imshow("90", rotate_big_edit)
cv2.waitKey()
cv2.destroyAllWindows()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:973: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


: 

ValueError: operands could not be broadcast together with shapes (143,143,3) (143,143,3) (181,181,3) 

In [ ]:
import cv2
import numpy as np

# 이미지 불러오기
img = cv2.imread(r'C:\ex\cvdir\insightbook.opencv_project_python-master\img\gaussian_noise.jpg')

# 이미지 크기
rows, cols = img.shape[:2]

# 회전 각도 및 배율 파라미터
angle_step = 5      # 각 프레임마다 회전할 각도
scale_initial = 0.5 # 초기 배율
scale_step = 0.01   # 배율 변화량

# 무한 루프를 통해 계속 이미지 회전
while True:
    for angle in range(0, 360, angle_step):
        # 회전 행렬 계산
        scale = scale_initial + (scale_step * angle) % 1
        mtrx = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, scale)
        
        # 이미지 회전 적용
        rotated_img = cv2.warpAffine(img, mtrx, (cols, rows), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
        
        # 결과 이미지 표시
        cv2.imshow('Rotated Image', rotated_img)
        
        # 키 입력 대기 (50ms)
        if cv2.waitKey(50) & 0xFF == 27:  # ESC 키를 누르면 종료
            cv2.destroyAllWindows()
            break


KeyboardInterrupt: 